# 📘 RAG Application (DMP Example) using LangChain + **Local Llama (Ollama)** + FAISS

This notebook is the same practical RAG tutorial, but **without OpenAI**.

✅ We use:
- **Ollama** to run a local Llama model for generation (e.g., `llama3.2`, `llama3.1`, etc.)
- **Ollama embeddings** for vector search (e.g., `nomic-embed-text` or `mxbai-embed-large`)
- **FAISS** as the vector store

You will:
1. Create a small DMP guidance file (`dmp_guidance.txt`)
2. Load it
3. Chunk it
4. Embed + index with FAISS
5. Build:
   - **Baseline (no RAG)**
   - **RAG (retrieve + grounded answer)**
6. Compare results for the same DMP question

---
## 🔴 Important
You must have **Ollama installed and running** on your machine.

- Install Ollama: https://ollama.com
- Then pull models from the terminal (or from notebook cells below):
  - `ollama pull llama3.2`
  - `ollama pull nomic-embed-text`  *(embedding model)*

## 🔧 1) Install Python dependencies (run once)
! pip install -r requirements.txt

## 🦙 2) Make sure Ollama is installed + pull models

Run these in your terminal **once** (recommended):
```bash
ollama pull llama3.2
ollama pull nomic-embed-text
```

Or you can try running the commands below from the notebook.  
(If your environment doesn't allow shell commands, do it in the terminal.)

In [3]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader


# --- Choose models you have pulled via `ollama pull ...`
LLM_MODEL = "llama3.2"            # generation model
EMBED_MODEL = "nomic-embed-text"  # embeddings model (recommended)

# Initialize Ollama chat model
llm = ChatOllama(model=LLM_MODEL, temperature=0.2)

# Initialize Ollama embeddings model
embeddings = OllamaEmbeddings(model=EMBED_MODEL)

print("✅ Ollama models configured:", LLM_MODEL, "|", EMBED_MODEL)

✅ Ollama models configured: llama3.2 | nomic-embed-text


## 📄 3) Create a practical DMP guidance document

For a blog/tutorial example, we create a small text file containing NIH DMS plan requirements and common guidance.
In production, you would replace this with:
- NIH policy pages (HTML/PDF)
- DMPTool guidance text
- Institutional sharing policy text
- Repository documentation

The key idea: **RAG retrieves from these sources** instead of the LLM guessing.


In [4]:
guidance_text = """
NIH DMS Plan Required Elements:
A Data Management and Sharing (DMS) Plan should address:
(1) Data type and amount
(2) Related tools, software, and documentation
(3) Standards
(4) Repository selection
(5) Timing of sharing
(6) Access, distribution, or reuse considerations

Repository selection guidance:
Repository selection should be justified by data type and community norms.
The repository should support metadata, persistent identifiers, long-term preservation,
and (when needed) access controls for sensitive data.

Human participant data & controlled access:
For human participant data, sharing may require controlled access and safeguards.
Plans should describe de-identification steps, governance (e.g., Data Use Agreements),
and an access review/approval process.

Documentation & metadata:
Documentation should support reuse: README, data dictionary/codebook, variable definitions,
provenance notes, and metadata needed for discovery and interpretation.

Timing of sharing:
Timing should specify when data will be shared (e.g., at publication, end of project,
or rolling releases). Any embargo/delay should be justified by ethics or policy.

Derived data when raw data cannot be shared:
If raw patient-level data cannot be shared broadly, plans can share derived or aggregated outputs
(e.g., feature tables, summary statistics, model performance metrics) to maximize reuse while protecting privacy.
""".strip()

with open("dmp_guidance.txt", "w", encoding="utf-8") as f:
    f.write(guidance_text)

print("✅ Created file: dmp_guidance.txt")

✅ Created file: dmp_guidance.txt


## 📥 4) Load the document using LangChain

LangChain represents documents as `Document` objects with:
- `page_content` (text)
- `metadata` (optional)

We’ll use `TextLoader` for this demo.

In [8]:
loader = TextLoader("dmp_guidance.txt", encoding="utf-8")
documents = loader.load()

print("Loaded documents:", len(documents))
print("\n--- Preview (first 600 chars) ---\n")
print(documents[0].page_content[:600] + "...")

Loaded documents: 1

--- Preview (first 600 chars) ---

NIH DMS Plan Required Elements:
A Data Management and Sharing (DMS) Plan should address:
(1) Data type and amount
(2) Related tools, software, and documentation
(3) Standards
(4) Repository selection
(5) Timing of sharing
(6) Access, distribution, or reuse considerations

Repository selection guidance:
Repository selection should be justified by data type and community norms.
The repository should support metadata, persistent identifiers, long-term preservation,
and (when needed) access controls for sensitive data.

Human participant data & controlled access:
For human participant data, sharin...


## ✂️ 5) Chunk the text

Why chunk?
- Embeddings work best on moderate-size text segments
- Retrieval becomes more precise

We use `RecursiveCharacterTextSplitter` to chunk with overlap.

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=60
)

chunks = text_splitter.split_documents(documents)

print("Total chunks:", len(chunks))
print("\n--- Example chunk ---\n")
print(chunks[0].page_content)

Total chunks: 4

--- Example chunk ---

NIH DMS Plan Required Elements:
A Data Management and Sharing (DMS) Plan should address:
(1) Data type and amount
(2) Related tools, software, and documentation
(3) Standards
(4) Repository selection
(5) Timing of sharing
(6) Access, distribution, or reuse considerations


## 🔍 6) Build FAISS vector store + retriever (with Ollama embeddings)

- Embed each chunk
- Store in FAISS
- Create retriever (`k=4`)

In [10]:
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

print("✅ Vectorstore ready (FAISS). Retriever k=4")

✅ Vectorstore ready (FAISS). Retriever k=4


## 🧠 8) Build Baseline vs RAG pipelines (no OpenAI)

We’ll do it in a simple, explicit way (easy to understand for a blog post).

### Baseline (no RAG)
- Just ask the Llama model directly

### RAG
- Retrieve top-k relevant chunks
- Inject them into the prompt
- Ask model to cite chunk IDs



In [13]:
def format_docs(docs):
    return "\n\n".join([f"[Chunk {i}] {d.page_content}" for i, d in enumerate(docs)])

def answer_without_rag(question: str) -> str:
    prompt = f"""You are helping write an NIH Data Management and Sharing Plan.
Answer the question using only your general knowledge.
Be concise and structured.

Question: {question}
"""
    return llm.invoke(prompt).content

def answer_with_rag(question: str) -> str:
    retrieved = retriever.invoke(question)
    context = format_docs(retrieved)

    prompt = f"""You are helping write an NIH Data Management and Sharing Plan.

Use ONLY the retrieved context to answer. If the context is insufficient, say what is missing.
Keep it practical and aligned with NIH DMS Plan language.

Question: {question}

Retrieved Context:
{context}

Answer format:
- Repository selection
- Timing of sharing
- Access / distribution / reuse
- Documentation & metadata
- Citations: (list chunk ids you relied on)
"""
    return llm.invoke(prompt).content

print("✅ Functions ready: answer_without_rag(), answer_with_rag()")

✅ Functions ready: answer_without_rag(), answer_with_rag()


## ✅ 9) Run the same question: WITHOUT RAG vs WITH RAG

This question matches our DMP-Chef style:
- human subjects
- restricted raw sharing
- need repository + timing + access controls language

In [14]:
question = (
    "Our project uses de-identified EHR notes and CT imaging. We will not share raw patient-level data broadly. " 
    "What should we write in the NIH DMS Plan about repository selection, timing of sharing, and access controls?"
)

print("\n" + "="*22 + " WITHOUT RAG " + "="*22)
print(answer_without_rag(question))

print("\n" + "="*24 + " WITH RAG " + "="*24)
print(answer_with_rag(question))


====================== WITHOUT RAG ======================
Here's a suggested outline for the NIH Data Management and Sharing (DMS) Plan:

I. Repository Selection

* Describe the de-identified EHR notes and CT imaging data repositories used to store and manage the data.
* Explain how the data will be stored in accordance with HIPAA regulations, such as using secure, encrypted, and auditable storage solutions.

II. Timing of Sharing

* Specify that raw patient-level data will not be shared broadly, but de-identified data may be shared through approved channels (e.g., Data & Knowledge Network).
* Describe the timeline for sharing de-identified data, including any milestones or deadlines.
* Explain how access to shared data will be restricted to authorized personnel and institutions.

III. Access Controls

* Outline the measures taken to ensure secure access to shared data, such as:
 + User authentication and authorization
 + Role-based access controls
 + Data encryption and anonymization

## Notes (practical tips)

- **Embeddings model matters**: `nomic-embed-text` and `mxbai-embed-large` are common choices in Ollama.
- **Llama is for generation**, not embeddings. Use a dedicated embedding model for retrieval.
- If your model is slow on CPU, try a smaller Llama variant (e.g., `llama3.2:3b` if available in the Ollama library).